In [4]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

%cd /content/drive/My\ Drive/
%cd broutanlab_course/

Mounted at /content/drive
/content/drive/My Drive
/content/drive/My Drive/broutanlab_course


In [3]:
pip install facenet-pytorch

     |████████████████████████████████| 1.9MB 11.0MB/s 


In [5]:
from facenet_pytorch import MTCNN, InceptionResnetV1
import torch
from torch.utils.data import DataLoader
from torchvision import datasets
import numpy as np
import pandas as pd
import os


In [6]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))


Running on device: cuda:0


In [7]:
#Define MTCNN module
mtcnn = MTCNN(
    image_size=160, margin=0, min_face_size=20,
    thresholds=[0.6, 0.7, 0.7], factor=0.709, post_process=True,
    device=device
)

In [8]:
#Define Inception Resnet V1 module

resnet = InceptionResnetV1(pretrained='vggface2').eval().to(device)

In [24]:
#Define a dataset and data loader

def collate_fn(x):
    return x[0]

dataset = datasets.ImageFolder('mini_dataset3')
dataset.idx_to_class = {i:c for c, i in dataset.class_to_idx.items()}
loader = DataLoader(dataset, collate_fn=collate_fn, num_workers=2)

In [25]:
aligned = []
names = []
for x, y in loader:
    x_aligned, prob = mtcnn(x, return_prob=True)
    if x_aligned is not None:
        print('Face detected with probability: {:8f}'.format(prob))
        aligned.append(x_aligned)
        names.append(dataset.idx_to_class[y])

Face detected with probability: 0.999995
Face detected with probability: 0.999972
Face detected with probability: 0.999844
Face detected with probability: 0.999969
Face detected with probability: 0.993608


In [26]:
aligned = torch.stack(aligned).to(device)
embeddings = resnet(aligned).detach().cpu()

In [27]:
dists = [[(e1 - e2).norm().item() for e2 in embeddings] for e1 in embeddings]
print(pd.DataFrame(dists, columns=names, index=names))

               Ariana Grande  Ariana Grande  ...  Selena Gomez   Unknown
Ariana Grande       0.000000       0.514325  ...      1.273568  1.410089
Ariana Grande       0.514325       0.000000  ...      1.292539  1.385727
Selena Gomez        1.343095       1.342803  ...      0.596805  1.336874
Selena Gomez        1.273568       1.292539  ...      0.000000  1.356669
Unknown             1.410089       1.385727  ...      1.356669  0.000000

[5 rows x 5 columns]
